<a href="https://colab.research.google.com/github/andrewdk1123/KoSentiment/blob/main/KoGPTDataAugmentor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load packages

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
import string
import random
import io
from transformers import AutoTokenizer, TFGPT2LMHeadModel, BertTokenizer
from nltk.tokenize import sent_tokenize

In [10]:
import nltk

nltk.download('punkt')  # Download the punkt tokenizer models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load data to be augmented

In [11]:
from google.colab import files

uploaded = files.upload()

Saving pos_training.csv to pos_training (2).csv


In [12]:
print(uploaded.keys())

dict_keys(['pos_training (2).csv'])


In [13]:
pos_training_data = pd.read_csv(io.BytesIO(uploaded['pos_training (2).csv']))
pos_training_data.head()

,Unnamed: 0,emotion,sentence,label,tokenized_sentence,cleaned_tokens
0,7,E66,퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.,1,"['퇴', '##사한', '지', '얼마', '안', '됐', '##지만', '천천...","['퇴', '지', '얼마', '안', '됐', '천천히', '직장', '구해']"
1,9,E66,요즘 직장생활이 너무 편하고 좋은 것 같아!,1,"['요즘', '직장', '##생활', '##이', '너무', '편하고', '좋은',...","['요즘', '직장', '너무', '편하고', '좋은', '것', '같아']"
2,10,E66,취업해야 할 나이인데 취업하고 싶지가 않아.,1,"['취업', '##해야', '할', '나이', '##인데', '취업', '##하고'...","['취업', '할', '나이', '취업', '싶지', '않아']"
3,22,E64,우리 회사는 정말 사내 분위기가 좋아.,1,"['우리', '회사는', '정말', '사내', '분위기가', '좋아', '.']","['우리', '회사는', '정말', '사내', '분위기가', '좋아']"
4,23,E64,오늘 내가 다니는 회사가 참 좋은 직장이라는 생각이 들었어.,1,"['오늘', '내가', '다니는', '회사가', '참', '좋은', '직장', '#...","['오늘', '내가', '다니는', '회사가', '참', '좋은', '직장', '생..."


In [14]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 30 23:47:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Augment sentences

KoGPT-2의 Language Model을 이용해서 긍정적인 문장을 생성합니다. 다음의 두 가지 가정으로부터 생성된 문장들이 긍정적인 감정을 나타낸다고 판단하였습니다.

1. 텍스트 문장에 포함된 각 단어는 긍정적인지 부정적인지의 감정을 전달한다.
2. 긍정적인 감정을 나타내는 단어들은 긍정적인 감정의 다른 단어들과 함께 나타날 가능성이 높다.

생성하는 시퀀스의 최대 길이는 50으로 하였고, Seed Sentence를 입력 하였을 때, 매 시점마다 Top 5개의 단어들 (top_k=5) 중에서 랜덤으로 선택하는 방식을 택하여 문장을 생성하였습니다.

In [15]:
def generate_sentence_with_top_k(seed_sentence, model, tokenizer, max_length=50, top_k=5):
    input_ids = tokenizer.encode(seed_sentence)

    while len(input_ids) < max_length:
        output = model(tf.convert_to_tensor([input_ids]))

        # Get the top k indices
        top_k_indices = tf.math.top_k(output.logits[0, -1], k=top_k).indices.numpy()

        # Randomly select the next token from the top k
        selected_token_id = random.choice(top_k_indices)
        input_ids.append(selected_token_id)

    generated_sentence = tokenizer.decode(input_ids)
    return generated_sentence

def main(df):
    model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
    tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

    generated_sentences = []

    for index, row in df.iterrows():
        seed_sentence = row['sentence']
        generated_sentence = generate_sentence_with_top_k(seed_sentence, model, tokenizer)
        generated_sentence = generated_sentence.replace(seed_sentence, '')
        generated_sentences.append({
            'emotion': row['emotion'],
            'generated_sentence': generated_sentence
        })
        print(f"Seed Sentence: {seed_sentence}\nGenerated Sentence: {generated_sentence}\n")

    generated_df = pd.DataFrame(generated_sentences)
    generated_df.to_csv('augmented_df.csv')
    files.download('augmented_df.csv')

Google colab에서 disconnected 되는 경우에 대비해서 전체 파일을 10등분 해서 각각 약 600여개의 문장으로 새로운 문장을 생성하겠습니다.

In [16]:
chunk1 = pos_training_data.iloc[:601,:]
chunk2 = pos_training_data.iloc[601:1201,:]
chunk3 = pos_training_data.iloc[1201:1801,:]
chunk4 = pos_training_data.iloc[1801:2401,:]
chunk5 = pos_training_data.iloc[2401:3001,:]
chunk6 = pos_training_data.iloc[3001:3601,:]
chunk7 = pos_training_data.iloc[3601:4201,:]
chunk8 = pos_training_data.iloc[4201:4801,:]
chunk9 = pos_training_data.iloc[4801:5401,:]
chunk10 = pos_training_data.iloc[5401:,:]

In [ ]:
# Apply the main function to each chunk
main(chunk1)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight', 'transformer.h.1.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.
Generated Sentence:  그 동안 힘들고 힘들어도 잘 이겨냈으니까. 그래서 다시 한번 더 기회를 <unk> 주셔서, 이 자리에 다시 오도록 해요. 이젠 더 잘 할 거 같아요.

Seed Sentence: 요즘 직장생활이 너무 편하고 좋은 것 같아!
Generated Sentence:  #아기띠 가 필요없다
#유아띠 가 좋아서 #맘스터치 를 검색해서 봤더니
아기가 있는 엄마들도 #육아맘 으로도

Seed Sentence: 취업해야 할 나이인데 취업하고 싶지가 않아.
Generated Sentence: .
어릴 때 공부해서 대학에 갈려고 했는데 취업도 못 하고.
그러면 어떻게 해야 되나요??
이렇게 해서 저도 모르게 이 일을 해 보고 싶은 생각이 들어요.

Seed Sentence: 우리 회사는 정말 사내 분위기가 좋아.
Generated Sentence:  내가 회사 내 분위기가 좋은데 왜 그러냐. 너가 회사에 대해 아는 바가 없느냐."
그는 말을 끊고는 다시 말했다.
이제까지 회사 내에서 가장 많은 사람이 회사를 떠난 뒤였다.
그렇지만 그가 떠난 뒤에는

Seed Sentence: 오늘 내가 다니는 회사가 참 좋은 직장이라는 생각이 들었어.
Generated Sentence:  그게 그 회사의 가장 중요한 가치라고 생각하고 있어.
내가 이 회사에서 일하는 게 가장 큰 보람이고 내가 이 회사의 가장 큰 보람이라고 생각하는 거 같더라.
그 회사에 다니는 게 내가

Seed Sentence: 회사에서 전공시험을 봤어. 오늘 시험 결과가 나왔어.
Generated Sentence:  그래서 내가 이번에 이번에 수강한 전공과 관련된 책을 한권 사게 된 건. 내가 이제 전공시험을 보는데 이번에 수능에서 봤던 전공의들이 좀

Seed Sentence: 오늘 입사 면접을 봤어.
Generated Sentence:  ᄏ"
이런 말을 들었는지 면접관들은 당황하는 표정으로 입을 다물지 못

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
main(chunk2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'lm_head.weight', 'transformer.h.5.attn.masked_bias', 'transformer.h.7.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 이번에 상견례를 하게 되었는데 예비 시부모님께서 두 분 다 친절하고 상냥하신 데다가 신세대처럼 재밌는 분들이라 너무 좋았어. 긴장하고 갔는데 너무 다행인 거 있지?
Generated Sentence: "며 웃었다.

Seed Sentence: 난 학점이 좋아서 취직이 잘 될 것 같아.
Generated Sentence:  그래서 취업할 때까지 기다렸지. 하지만 그땐 내가 정말 잘했거든. 그때 내가 얼마나 많이 배웠는지, 내가 어떻게 해서 이겼는지도 모르겠어! 내가 왜 이런 걸까?

Seed Sentence: 이번에 원하는 기업에 취업 성공했어. 취업 준비 기간이 길어질까 봐 걱정했는데 너무 다행이야.
Generated Sentence:  내가 원하는 기업을 알아보고 싶었지. 그 기업이 얼마나 잘될까 궁금해서, 또 다른 기업에 취업해서 성공하는 게 어

Seed Sentence: 나한테 여자친구가 생겼어! 선물하고 싶은데 어떤 게 좋을까?
Generated Sentence:  그 여자 친구랑 만나면 좋을 텐데"
나는 그렇게 말했더니 그는 이렇게 말했다.
그는 나를 쳐내듯이 말했다.
나는 그의 얼굴을 쳐보았다.
그리고는 다시 말했다.
그

Seed Sentence: 요즘 직장 상사 때문에 걱정이야.
Generated Sentence:  그건 내가 책임질 일이 아닌 것 같군.
"아뇨. 뭐라고 하세요?
이제 그만 하세요.
그건 내가 알아서 해. 그건 내일 아침에 일어나.
그런 식으로 해

Seed Sentence: 얼마 전에 친구랑 싸워서 걱정이 많았어.
Generated Sentence:  그런데 오늘 보니 내가 너무 잘해줬구나 하는 느낌이 들었던 것 같아.” 그는 “이제야 내가 잘해줬다고 느낀 것뿐이지”라고 말했다.
그날 밤 친구와 술자리를 가진

Seed Sentence: 요즘 직장 다니기 힘들어.
Generated Sentence:  그래서 이걸 또 어떻게든 이거를 이케 이걸 또 이걸 어르신에게 알려야 되는데 이걸 좀 알려야 되는데요

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
main(chunk3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 남편이 술을 너무 자주 마셔서 걱정인데 본인은 건강하다고 자신하네.
Generated Sentence: "
나는 이렇게 말하면서도 뭔가 다른 것을 느끼고 있다.
"그렇다. 술 마시는 거는 건강에 좋지 않다는 얘기잖냐."
그때 갑자기 한 여자가 들어왔다.


Seed Sentence: 남편하고 나하고 열심히 저축해서 남편이 은퇴해도 별걱정 없어.
Generated Sentence:  그걸 내가 어떻게 알아.
이건 정말 중요한 일이야, 나 혼자만 그런 거야. 나하고 결혼하는 것도 아니고 결혼하고 나니까 정말 기분이 좋은데 왜

Seed Sentence: 친구가 나를 많이 도와줘서 좋아
Generated Sentence: 요~! ᄏ ᄏ</d> 저번에 ᄏᄏ
이번에 또 한번
이제부터 #오픈마켓에서 #쿠쿠쿠를 만나봅니다
<unk>
이렇게 다양한

Seed Sentence: 나 임신하게 되었어 축하해줄래?
Generated Sentence: "라는 멘트가 흘러나온다.
그리고 이 멘트는 이걸로도 충분하다는 의견이 많다.
아직 임신중인데다 출산 후 스트레스가 많이 쌓였기 때문이라고.</d> 이게 무슨 짓인가 싶어서 ( 

Seed Sentence: 나 있지 결혼을 너무 잘한 것 같아. 남편이 너무 잘해줘.
Generated Sentence:  그거 내가 다 알아서 해줄 테니깐, 내가 잘 해줘."
아무튼 그녀는 남편의 손을 잡고는 남편의 팔을 붙잡고는 말했다.
그녀의 눈빛에는 미

Seed Sentence: 결혼을 하고 이렇게 행복할 줄 몰랐어.
Generated Sentence:  아빠가 그거 다 알고 있는 것 아니에. 엄마는 그~ 내가 이걸 알잖어? 아빠랑 내가 아는 게 다 알고 있었어."
나는 아~ 그걸 알면서도

Seed Sentence: 대학교에서 마음에 드는 친구가 나에게 먼저 말을 걸었어.
Generated Sentence:  그리고 그게 내가 좋아하는 거니까, 나는 그 친구를 따라다녔어."
그런 식으로 말했을 때의 그 순간의 기억을 나는

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
main(chunk4)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 이번에 운 좋게 수출 계약서에 사인을 받았어.
Generated Sentence:  그건 네네. 그렇군요.
예예 예예예. 자 그러면 이제 이제 우리 이제 우리 수출보험공사는 수출에 대해서 좀 적극적으로 지원을 해주겠다라는 약속을 드리고 있는데 이

Seed Sentence: 남자 친구와 있으면 마음이 너무 편안해져.
Generated Sentence:  내가 너희들에게 무슨 짓이라도 하면 그땐 너희가 용서해주마”라고 했다.
그런데 그 친구가 ‘나는 왜 그런 일을 했느냐’며 화를 내는 것이었다.
그러자 “너는 너희에게

Seed Sentence: 외국에 계시는 이모님이 결혼 선물을 보내 주셨어.
Generated Sentence:  내가 결혼해서 잘 살게 됐어요. 그런데 이모는 결혼할 마음이 전혀 없었거든? 이모와 내가 결혼해서 살 거면 어떡해? 결혼 안 할 거니까? 내가 살

Seed Sentence: 나 이번에 회사에서 승진했어. 승진하면서 연봉도 많이 오르게 될 것 같아.
Generated Sentence:  그래서 그 회사에서 내가 뭘 해야 할지를 모르겠는데 내가 지금 이 회사를 그만두면서 이 회사에서는 괜한 걱정을 할 필요 없어.
이런 식으로

Seed Sentence: 드디어 남자 친구랑 결혼해.
Generated Sentence:  난 남자 친구랑 결혼하면 어차피가 더 좋겠는데.
그래. 너랑 결혼해?
아뇨? 아니요.
결혼하면 어때요~ 결혼하고 싶어요?
결혼할 거 같으면 그냥 그냥 그냥 결혼

Seed Sentence: 퇴직하면 세계 여행이나 가야겠어.
Generated Sentence:  그러고 나서 그 다음에 여행이 좀 쉬어지면 좋겠다는 생각을 한 적이 있더군." 그 후로는 계속 여행하고 싶어.
여행할 때마다 꼭 해야만 할 일을 찾느라 애쓰지만,

Seed Sentence: 은퇴하면 책을 하나 쓸 거야.
Generated Sentence:  그래서 책을 쓸 때 가장 좋은 게 바로 책 읽기인데 책을 읽는다고 책을 읽는다는 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
main(chunk5)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 아직도 주말이면 산 정상에 오를 정도로 체력이 좋아. 내가 체력만큼은 좋다고 자부해.
Generated Sentence:  그래서 내일도 내가 좋아하는 운동복을 입을게요."
이런 말을 들은 김씨는 고개를 돌려 고개를 돌려 나를 올려보았다.
나는 고개를 끄적끄

Seed Sentence: 운동을 안 해도 지금껏 아주 건강히 잘 살고 있어.
Generated Sentence:  그리고 그게 바로 그 사람이 하는 일이지."
"아니, 내가 그 사람이 하는 일이니까 내가 어떻게 해 줄 거니! 그건 네 몫이거든! 내가 알아서 할테니

Seed Sentence: 언제 떠나도 이 생에 미련이 남지 않도록 늘 최선을 다해 살아왔어.
Generated Sentence:  그런데 내가 왜 이 생에 이 미련은 남지 않게 되었는지 그 이유는 넌 알지 못해? 난 너에게 미련이란 없어져. 내가 너

Seed Sentence: 모두 내 나이를 알면 깜짝 놀라며 건강의 비법이 무엇이냐고 자주 물어봐.
Generated Sentence:  난 그걸 알지도 못했어!" 라고 말한다.
그러자 이젤의 말에 이젤도 웃으며 웃으며 대답한다.
"이제야 겨우

Seed Sentence: 젊었을 때 성질에 비해 지금은 아주 여유로워.
Generated Sentence:  그래서 나도 이번에 이~ 우리 저~ 저도 그~ 우리 이거 뭐 저도 이~ 그~ 그거 저~ 이~ 저희도 뭐~ 이거 저도 저

Seed Sentence: 남편의 당뇨 때문에 운동 겸 등산을 다니면서 좋은 풍경을 보고 와.
Generated Sentence:  그래서 내가 당뇨를 치료하려고 하면 내가 더 잘 알 것 같다고 했거든요.
그래, 그래서 당뇨병도 좀 잘 관리를 하기로 결심했어요.
그리고 나중에도 내가 잘 관리를 하면

Seed Sentence: 며느리가 날 간병하게 돼서 모녀처럼 지내려고 노력 중이야.
Generated Sentence:  그건 그렇다 치고. 그런데 내가 이럴 수 있는 게 이거 너무 힘들게 해서 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
main(chunk6)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 요새 자식들이 나한테 신경을 굉장히 많이 써줘서 기분이 아주 좋아.
Generated Sentence:  그니까 뭐가 그리 중요해서가 아니라, 이~ 저런 얘기를 많이 하는 게 아니고 그걸로 그거를 그게 그걸로 이제 뭐야

Seed Sentence: 정말 인생 최고의 날이야. 내 자식들이 여행을 보내줬거든!
Generated Sentence:  내 인생에 최고의 여행이었으니까.
그래. 내 자식이 여행 가는데 여행가방을 들고 왔었거든! 그걸 갖고 갔는데 그거 때문에 내 아들도 비행기 타고

Seed Sentence: 내 자식들이 글쎄 지들끼리 돈을 모아서 좋은 곳으로 날 이사시켜 줬지 뭐야? 정말 너무 뿌듯하다.
Generated Sentence:  내가 이걸 어떻게 해볼 수 있을까?"
나는 이렇게 말하면서 고개를 끄떡이며 고개를 저으며 이렇게 말하고 싶

Seed Sentence: 젊어서 고생은 사서도 한다는 말이 맞았어. 괜스레 기분이 좋네.
Generated Sentence:  뭔가가 너무 좋아졌거든?"
아무렇지 않게 대답했다.
아니면 넌 내일 아침 일찍 잠에서 깨어날까 하고

Seed Sentence: 다행이다! 병원비가 생각보다 적게 나왔어.
Generated Sentence:  그건 내가 감당할 수 있을지. 뭘 어떻게 하라는 말이야, 그걸 내가 어떻게 해야 할지. 내가 왜 이 지경을 만들었지?
이제부터가 문제야

Seed Sentence: 다행이야. 내 병이 노인혜택을 받을 수 있는 질병이래.
Generated Sentence:  이걸 어떻게든 해볼까."
"아니. 그걸 내가 왜 해줄 수가 없잖아."
"그래? 이럴 수가 없단다. 이거야

Seed Sentence: 참 미안하면서도 고마운 일이야. 병원비를 동생들이 다 내줬어.
Generated Sentence:  내가 너무 잘해줘. 내 딸도 잘 컸고."
나는 다시 한번 고맙게 생각하고 말했다.
그리고 그 말에 나는 다시 가슴이 미어진다.
그리고 또

Seed Sentence: 우

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
main(chunk7)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 어버이날에 나에게 카네이션을 달아주는 아이들의 모습에 보람을 느껴.
Generated Sentence:  우리 모두 카네시스를 선물해 봅니다.
오늘부터 매주 금·토요일을 이용해 어린이들의 눈높이 교육에 도움을 주는 카운슬링 프로그램을 시작했답니다.
우리

Seed Sentence: 아내가 언제나 나를 충분히 배려하고 이해해 줘서 좋아.
Generated Sentence:  내가 너무 좋아. 내가 정말 좋아하고 좋아하는 걸 내가 다 해 줄 수 있는 사람이지.
나는 정말 고맙고 사랑하고.
그렇게 생각하니까 너무 좋아서 행복하고

Seed Sentence: 일과 가정 둘 모두에 집중할 수 있는 지금의 삶이 정말 만족스러워.
Generated Sentence:  그땐, 그게 내 꿈이야.
그땐 정말이지 행복하더라고, 정말이지 그건 꿈이야.
그래도 그땐 내 꿈에 대한 믿음이 더 강

Seed Sentence: 아내와 나는 아이가 없어도 지금 충분히 만족해.
Generated Sentence:  내가 너한테서 얻은 것은 너와 나, 내 아들뿐이라구! 네 말이 맞다면 내가 널 얼마나 잘 돌보고 있나?
너를 잘 보살피고 있어!
나는 지금 이 순간,

Seed Sentence: 오늘 여자친구를 위해 어렵게 예약한 음식점이 듣던 대로 굉장히 만족스러웠어.
Generated Sentence:  그게 그게 제일 인상적이였지? 아유 정말 너무너프네요. 예. 그래서 이걸 또 다시 사야 되는데 아

Seed Sentence: 나는 아내와의 결혼 생활에 매우 만족하고 있어.
Generated Sentence:  그런데 이 여자가 왜 그렇게 많은 사랑을 받고 있을까?
왜냐하면, 그녀는 남편보다 먼저 결혼한 여자일지도 모름. 그리고 그녀가 사랑하는 여자는 그 여자가 사랑하는 여자와는 너무나 많은 시간을 같이 보내기 때문에

Seed Sentence: 아내와 세운 임신 계획이 아주 잘 맞아떨어져서 너무 만족스러워.
Generated Sentence:  그래서 내가 임신 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
main(chunk8)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 일흔의 나이까지 건강하게 살고 있는 나에게 자식들이 여행을 보내줬어.
Generated Sentence:  그리고 그분들은 내 인생 최고의 친구들인데."
그녀는 자신의 삶을 위해 최면을 건다고 말했다.
그리고 내 인생을 바꿀 만한 일이 있을 때 그분들을 위해 기도

Seed Sentence: 신체나이가 내 나이보다 훨씬 젊게 나왔어.
Generated Sentence:  내가 이걸로 내 인생을 살 수 있을까? 나는 이대로 죽는 걸 바라지 않을 테고"
"아니, 나는 죽는 게 싫어."
이렇게 말한 후 이젠 다시 그 말에

Seed Sentence: 유치원생인 손자의 체육대회에 손자와 함께 이인삼각 달리기를 하기로 했어.
Generated Sentence:  그래서 나는 이거 하고 놀았단다. 그리고 나서 이번에 우리 손자가 이겼단 말예요. 그래서 그걸 보면서 우리 손자는 이거

Seed Sentence: 체육센터에서 운동을 하는데 회원들이 나에게 젊어 보인다고 말했어. 아주 신이 나.
Generated Sentence:  그리고 나는 그 친구와 내가 같이 운동을 하는 모습을 보며 정말 신기했거든."
나는 운동할 때마다 항상 그 친구가 내게 좋은 친구가 되고 싶었어.


Seed Sentence: 딸이 내 건강을 위해 안마기를 사줬어! 너무 신나고 기뻐.
Generated Sentence:  그리고 내가 이젠 안아 줄 테니 너희들이랑 같이 살 수 있어."
아무래도 내 마음은 내 건강보다는 내가 건강해지지 않을까 싶어 마음이 급해진 모양이다.


Seed Sentence: 친했던 동창 친구들을 만났는데 모두 건강하게 살아있었어. 너무 기뻐.
Generated Sentence:  그리고 나랑 친해진 이후로 나는 항상 친하게 지내서 항상 친해져."
"친구들과 얘기하는 걸 보면 내가 친해진 게 정말 감사한 거

Seed Sentence: 내 나이가 칠십 세인데 이렇게나 건강해. 너무 기뻐.
Generated Sentence:  아이고. 그래? 예뻐

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
main(chunk9)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 내 나이에 이렇게 건강할 수 있음에 감사해.
Generated Sentence:  내일부터 내 몸에 대해 얘기를 할게! 내 몸에 관해 얘기는 해! 내 몸의 어떤 부분에 관해 말해 줄 거야, 이게 내 몸의 어떤 부분을 말하는 거구나!
내 몸

Seed Sentence: 나 조만간 수술을 해야한대.
Generated Sentence:  수술 후유증을 생각하면 가슴이 답이 없는 거 같아요.
수술을 하면 그걸 또 다시 되돌릴 방법이 없을까 걱정되는데요.
네. 수술 전엔 뭐 뭐라 그럴까 수술 후유증을 좀

Seed Sentence: 건강검진 결과가 나왔는데 건강하다고 해서 안심이야.
Generated Sentence:  건강하긴 하지만 그래도 이 정도면 좀 더 건강할 수 있을 거 같아요."
나는 고개를 흔들고 고개를 저으며 말했다.
그리고 나서 말했다.
그녀는 나를 바라보면서 내 옆에 있는

Seed Sentence: 아침 프로그램에서 석류즙이 그렇게 좋다는데. 사야겠다.
Generated Sentence:  네~ 그래서 사왔다면서요.
이렇게 해서 석류가 많이 들어있다는 걸 알게 되었다죠.
그래도 석류에 대해서는 뭐랄까.
그렇다라는 걸

Seed Sentence: 우리팀이 창업 아이디어 프로젝트에서 일등을 했어 그래서 포상금을 받았는데 기분이 좋아 그동안의 스트레스가 한번에 날아가는 것 같아.
Generated Sentence:  그리고 또 그걸 통해서 이제 이게 이제 또 이십대 후반이라는 나이까지 또 한

Seed Sentence: 이번 프로젝트 성공적이었다고 사장님께 격려금 받았어. 너무 기분 좋아.
Generated Sentence:  감사합니다.</d> 이 책은 한국경제에 대해 '긍정의 시대' 혹은 '대응시대'의 의미를 부여한다.
'긍정시대'란 '경제위기'를 뜻하는

Seed Sentence: 드디어 승진 대상자에 선정됐어! 너무 신나!
Generated Sentence: " 라는 말을 남기며 "너희들의 꿈, 너희는 나의 꿈을 이뤄줄 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
main(chunk10)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 그녀가 나랑 결혼하겠대. 너무 신나.
Generated Sentence:  그게 무슨 소용이야? 그건 내 잘못이지? 그게 아니라면 내 결혼은 포기할 거 아뇨. 내가 원하는 게 뭐예요. 내가 좋아하는 건."
이미 그녀는

Seed Sentence: 업무가 너무 과중하긴 한데 그래도 시작해봐야지.
Generated Sentence:  그걸 왜 이렇게 해야 하나?
그게 문제예요.
아니야! 아빠는 뭐? 아빠, 아줌마는 왜 이러시죠, 뭐. 엄마!
그냥

Seed Sentence: 회사에서 명절 보너스로 백 퍼센트 상여금이 나왔어. 생각지도 못한 보상이라 너무 기분이 좋아.
Generated Sentence:  이게 다 내가 줬던 것 같아요. 그걸로 내가 더 이상 못 뗄 거 같진 않았잖니. 내가

Seed Sentence: 오늘 가고 싶은 회사 면접을 봤어.
Generated Sentence:  어. 어.
그럼. 면접은 어떻게 하세요?</d> 요즘 뭐가 제일 핫해요
요거 뭘까요?
저희 집에선 <unk>가 진짜 진짜 진짜 이뻐요


Seed Sentence: 아빠가 오늘 엄마랑 엄청나게 싸웠는데 싸우는 도중에 이혼 얘기까지 나왔었어.
Generated Sentence:  아휴. 근가 엄마한테는 아빠가 아빠가 뭐라구 그랬냐고. 아유 그~ 이혼한 지 얼마 안 된 엄

Seed Sentence: 오늘도 예전과 같이 교실에서 학교폭력이 일어날 뻔했어. 그런데 선생님이 그 현장을 목격해서 너무 다행이야.
Generated Sentence:  그래서 그 현장에 있던 선생님을 보고 그 현장에 있었던 선생님을 뵈러 왔었대요. 그래서 제가

Seed Sentence: 원하던 직장에 합격해서 좋다.
Generated Sentence: "고 말했다.
이어 김 씨는 “대학입시를 위해 학자금 마련이 힘들고 취업이 잘되지 않는 상황에서, 대학은 취업 준비생들을 대상으로 한 맞춤 교육프로그램을 운영하고 있는 것” 이라면서 “

Seed Sentence: 결혼생활이 만족스러

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>